# GLASS model analysis

## Imports

In [1]:
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
gleam_root = "/Users/phdenzel/gleam"
sys.path.append(gleam_root)
from gleam.glass_interface import glass_renv
from gleam.utils.lensing import LensModel
from gleam.utils.lensing import kappa_profile, find_einstein_radius, inertia_tensor, qpm_abphi
from gleam.utils.plotting import viewstate_plots
glass = glass_renv()

In [2]:
states = !ls states/*.state
states

['states/SDSSJ0029-0055.state',
 'states/SDSSJ0737+3216.state',
 'states/SDSSJ0753+3416.state',
 'states/SDSSJ0956+5100.state',
 'states/SDSSJ1051+4439.state',
 'states/SDSSJ1430+6104.state',
 'states/SDSSJ1627-0053.state']

In [3]:
models = [LensModel(f) for f in states]
models

In [20]:
for m in models:
    print(m.obj_name)
    M_tot = []
    for i in range(-1, m.N):
        M = (np.sum(m.kappa*m.kappa_grid(model_index=i, refined=True)*m.pixel_size**2))/m.dlsds
        M_tot.append(M)
    M_min, M_max, M_ens = min(M_tot), max(M_tot), M_tot[0]
    print("M_tot [1e11 M_sol] = {:2.4f}_{{-{:2.4f}}}^{{+{:2.4f}}}".format(M_ens/1e11, abs(M_ens-M_min)/1e11, abs(M_ens-M_max)/1e11))
    print("R_M [arcsec] = {:1.4f}".format(m.maprad))
    # print("{:e}".format(m.env.ensemble_average['obj,data'][0][1]['M(<R)'][-1]))

SDSSJ0029-0055
M_tot [1e11 M_sol] = 1.4817_{-0.0911}^{+0.0996}
R_M [arcsec] = 1.2746
SDSSJ0737+3216
M_tot [1e11 M_sol] = 4.0940_{-0.2697}^{+0.2736}
R_M [arcsec] = 1.4571
SDSSJ0753+3416
M_tot [1e11 M_sol] = 1.4669_{-0.0675}^{+0.0733}
R_M [arcsec] = 1.6240
SDSSJ0956+5100
M_tot [1e11 M_sol] = 5.3378_{-0.3415}^{+0.3513}
R_M [arcsec] = 1.9437
SDSSJ1051+4439
M_tot [1e11 M_sol] = 2.9470_{-0.2029}^{+0.1982}
R_M [arcsec] = 1.9874
SDSSJ1430+6104
M_tot [1e11 M_sol] = 1.8337_{-0.1460}^{+0.1786}
R_M [arcsec] = 1.6287
SDSSJ1627-0053
M_tot [1e11 M_sol] = 3.0916_{-0.1407}^{+0.2111}
R_M [arcsec] = 1.7368


In [5]:
# Calculate all kappa profiles
kappa_profiles = {}
for m in models:
    print(m.obj_name)
    kappa_profiles[m.obj_name] = []
    for i in range(-1, m.N):
        kp = kappa_profile(m, mdl_index=i)
        kappa_profiles[m.obj_name].append(kp)

SDSSJ0029-0055
SDSSJ0737+3216
SDSSJ0753+3416
SDSSJ0956+5100
SDSSJ1051+4439
SDSSJ1430+6104
SDSSJ1627-0053


In [6]:
%%script false
r, kp = kappa_profiles['SDSSJ0029-0055'][0]
plt.plot(r, kp)
plt.show()

In [7]:
# Calculate Einstein radii from profiles
einstein_radii = {}
for m in models:
    print(m.obj_name)
    einstein_radii[m.obj_name] = []
    profiles = kappa_profiles[m.obj_name]
    for i, (r, kp) in enumerate(profiles):
        r_E = find_einstein_radius(r, kp)
        einstein_radii[m.obj_name].append(r_E)
    r_min = min(einstein_radii[m.obj_name])
    r_max = max(einstein_radii[m.obj_name])
    r_ens = einstein_radii[m.obj_name][0]
    print("R_Einstein = {:4.4f}_{{-{:4.4f}}}^{{+{:4.4f}}}".format(r_ens, abs(r_ens-r_min), abs(r_ens-r_max)))

SDSSJ0029-0055
R_Einstein = 0.9657_{-0.0371}^{+0.0352}
SDSSJ0737+3216
R_Einstein = 1.0252_{-0.0488}^{+0.0501}
SDSSJ0753+3416
R_Einstein = 1.3137_{-0.0427}^{+0.0371}
SDSSJ0956+5100
R_Einstein = 1.3956_{-0.0512}^{+0.0502}
SDSSJ1051+4439
R_Einstein = 1.4956_{-0.0652}^{+0.0650}
SDSSJ1430+6104
R_Einstein = 1.1528_{-0.0625}^{+0.0933}
SDSSJ1627-0053
R_Einstein = 1.2667_{-0.0320}^{+0.0315}


In [8]:
# Calculate quadrupole
abphi = {}
for m in models:
    print(m.obj_name)
    abphi[m.obj_name] = [[], [], [], []]
    for i in range(-1, m.N):
        kgrid = m.kappa_grid(model_index=i, refined=True)
        px = m.pixel_size
        qpm = inertia_tensor(kgrid, pixel_scale=px, com_correct=False)
        a, b, phi = qpm_abphi(qpm)
        abphi[m.obj_name][0].append(a)
        abphi[m.obj_name][1].append(b)
        abphi[m.obj_name][2].append(phi)
        abphi[m.obj_name][3].append(b/a)
    a_min, a_max, a_ens = min(abphi[m.obj_name][0]), max(abphi[m.obj_name][0]), abphi[m.obj_name][0][0]
    b_min, b_max, b_ens = min(abphi[m.obj_name][1]), max(abphi[m.obj_name][1]), abphi[m.obj_name][1][0]
    q_min, q_max, q_ens = min(abphi[m.obj_name][3]), max(abphi[m.obj_name][3]), abphi[m.obj_name][3][0]
    phi_min, phi_max, phi_ens = min(abphi[m.obj_name][2]), max(abphi[m.obj_name][2]), abphi[m.obj_name][2][0]
    phi_min *= 180/np.pi
    phi_max *= 180/np.pi
    phi_ens *= 180/np.pi
    print("a = {:4.4f}_{{-{:4.4f}}}^{{+{:4.4f}}}".format(a_ens, abs(a_ens-a_min), abs(a_ens-a_max)))
    print("b = {:4.4f}_{{-{:4.4f}}}^{{+{:4.4f}}}".format(b_ens, abs(b_ens-b_min), abs(b_ens-b_max)))
    print("q = {:4.4f}_{{-{:4.4f}}}^{{+{:4.4f}}}".format(q_ens, abs(q_ens-q_min), abs(q_ens-q_max)))
    print("\phi = {:4.4f}_{{-{:4.4f}}}^{{+{:4.4f}}}".format(phi_ens, abs(phi_ens-phi_min), abs(phi_ens-phi_max)))
    

SDSSJ0029-0055
a = 1.0316_{-0.0606}^{+0.0651}
b = 0.9521_{-0.0780}^{+0.0639}
q = 0.9229_{-0.0860}^{+0.0736}
\phi = 63.3156_{-49.2938}^{+71.3082}
SDSSJ0737+3216
a = 1.1800_{-0.0671}^{+0.0629}
b = 1.1226_{-0.0851}^{+0.0710}
q = 0.9513_{-0.0917}^{+0.0464}
\phi = 93.3654_{-91.3879}^{+85.8858}
SDSSJ0753+3416
a = 1.2620_{-0.0718}^{+0.0814}
b = 1.2356_{-0.0930}^{+0.0629}
q = 0.9790_{-0.0989}^{+0.0200}
\phi = 135.8942_{-135.8102}^{+44.0812}
SDSSJ0956+5100
a = 1.5595_{-0.0934}^{+0.1272}
b = 1.4964_{-0.1240}^{+0.0761}
q = 0.9595_{-0.1168}^{+0.0387}
\phi = 61.0554_{-59.7405}^{+118.2575}
SDSSJ1051+4439
a = 1.6253_{-0.1023}^{+0.1138}
b = 1.4869_{-0.1508}^{+0.1093}
q = 0.9148_{-0.0873}^{+0.0757}
\phi = 15.7759_{-15.7149}^{+164.2143}
SDSSJ1430+6104
a = 1.3045_{-0.0636}^{+0.0983}
b = 1.2889_{-0.0862}^{+0.0469}
q = 0.9880_{-0.1060}^{+0.0115}
\phi = 88.2828_{-86.8176}^{+91.5621}
SDSSJ1627-0053
a = 1.4169_{-0.0693}^{+0.0874}
b = 1.3008_{-0.0787}^{+0.0794}
q = 0.9181_{-0.0664}^{+0.0632}
\phi = 4.5037_{-4.